In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import load_model
import joblib
from sklearn.preprocessing import LabelEncoder, StandardScaler

import os
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

model_paths = {
    'pitchLine_avoid': 'Models/pitchLine_model_filtered.h5',
    'pitchLength_avoid': 'Models/pitchLength_model_filtered.h5',
    'pitchLine_safe': 'Models/pitchLine_model_safe.h5',
    'pitchLength_safe': 'Models/pitchLength_model_safe.h5',
    'Bowler_Bowling_Style_safe': 'Models/Bowler_Bowling_Style_model_safe.h5',
    'shotType': 'Models/shotType_model.h5'
}
models = {name: load_model(path) for name, path in model_paths.items()}

label_encoders = joblib.load("Models/label_encoders.pkl")
scaler = joblib.load("Models/scaler.pkl")

input_cols = ['inningNumber', 'oversActual', 'totalRuns', 'totalWickets',
              'time_of_day', 'Ground Name', 'Batsman_Name', 'Batsman_Role',
              'Batsman_Batting_Style', 'Batsman_Playing_Role']

user_input = {
    'inningNumber': 1,
    'oversActual': 20,
    'totalRuns': 150,
    'totalWickets': 3,
    'time_of_day': 'Day',
    'Ground Name': 'Wankhede Stadium',
    'Batsman_Name': 'Virat Kohli',
    'Batsman_Role': 'Batsman',
    'Batsman_Batting_Style': 'Right-Handed',
    'Batsman_Playing_Role': 'Top-Order'
}

df_input = pd.DataFrame([user_input])

for col, encoder in label_encoders.items():
    try:
        df_input[col] = encoder.transform(df_input[col])
    except ValueError:
        print(f"Invalid value for {col}. Expected one of: {list(encoder.classes_)}")
        exit()

numeric_cols = [col for col in df_input.columns if col not in label_encoders]
df_input[numeric_cols] = scaler.transform(df_input[numeric_cols])

print("\n--- Model Predictions ---")
for name, model in models.items():
    preds = model.predict(df_input)
    class_idx = np.argmax(preds[0])
    print(f"{name} => Class Index: {class_idx}")